* The idea of this exercise
* The idea of this exercise is to build a simple RAG pipeline with text intensive data
* We would use vertex embedding
* we would use FIASS vector store
* we would create a prompt for a student studying class 10 clarifying some doubts

In [6]:
# Indexing Pipeline
# 1. loader
from langchain_unstructured import UnstructuredLoader
from langchain_community.document_loaders import DirectoryLoader

In [2]:
# 2. splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
# 3. embeddings
from langchain_google_vertexai.embeddings import VertexAIEmbeddings

In [4]:
# 4. vectorstore
from langchain_community.vectorstores import FAISS

In [ ]:

document_loader = UnstructuredLoader("/Users/munna/VScode/QT-AgenticAI/RAG/hello-langchain/notebooks/concepts/*.txt")

In [8]:
# lets load documents

directory_loader = DirectoryLoader(
    path="/Users/munna/VScode/QT-AgenticAI/RAG/hello-langchain/notebooks/concepts",
    glob="*.txt",
    loader_cls=UnstructuredLoader,
)
documents = directory_loader.load()

In [9]:
# investigate documents
print(f"number of docs {len(documents)}")
print("\n\n*****************\n\n")
print(f"100 characters content of first doc \n{documents[0].page_content[100:]}")
print("\n\n*****************\n\n")
print(f"metadata of first doc \n{documents[0].metadata}")

number of docs 225


*****************


100 characters content of first doc 



*****************


metadata of first doc 
{'source': '/Users/munna/VScode/QT-AgenticAI/RAG/hello-langchain/notebooks/concepts/velocity.txt', 'last_modified': '2026-01-25T16:44:33', 'languages': ['eng'], 'file_directory': '/Users/munna/VScode/QT-AgenticAI/RAG/hello-langchain/notebooks/concepts', 'filename': 'velocity.txt', 'filetype': 'text/plain', 'category': 'Title', 'element_id': 'c0f318013c3bf97f3c879c571c464fa2'}


In [10]:
# chuncking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)

In [11]:
chunks = text_splitter.split_documents(documents)

In [12]:
# investigate chunks
chunks[4].metadata

{'source': '/Users/munna/VScode/QT-AgenticAI/RAG/hello-langchain/notebooks/concepts/velocity.txt',
 'last_modified': '2026-01-25T16:44:33',
 'languages': ['eng'],
 'file_directory': '/Users/munna/VScode/QT-AgenticAI/RAG/hello-langchain/notebooks/concepts',
 'filename': 'velocity.txt',
 'filetype': 'text/plain',
 'parent_id': 'c0f318013c3bf97f3c879c571c464fa2',
 'category': 'NarrativeText',
 'element_id': '65def5f8b428a8029f26ce1d80358cc3'}

In [30]:

embedding = VertexAIEmbeddings(
    model_name="text-embedding-005"
)


/var/folders/yx/b4_zr4nn6c701qx2kygmf_qc0000gn/T/ipykernel_8116/965166775.py:1: DeprecationWarning: Use [`GoogleGenerativeAIEmbeddings`][langchain_google_genai.GoogleGenerativeAIEmbeddings] instead.
  embedding = VertexAIEmbeddings(
/opt/anaconda3/lib/python3.13/site-packages/google/auth/_default.py:114: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [32]:
# store chunks into vectors in vector store
BATCH_LIMIT = 250
vector_store = FAISS.from_documents(
    chunks[:BATCH_LIMIT],
    embedding
)

# Remaining batches
for i in range(BATCH_LIMIT, len(chunks), BATCH_LIMIT):
    vector_store.add_documents(chunks[i:i + BATCH_LIMIT])

INFO: HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/munna-genai/locations/us-central1/publishers/google/models/text-embedding-005:predict "HTTP/1.1 200 OK"
INFO: Loading faiss.
INFO: Successfully loaded faiss.
INFO: HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/munna-genai/locations/us-central1/publishers/google/models/text-embedding-005:predict "HTTP/1.1 200 OK"


In [33]:
retriever = vector_store.as_retriever()

In [34]:
# Now we need to take a question from user
question = "What does Accelartion describe ?"
# question = "What is heat?"
# question = input("Ask your question")

In [35]:
relevant_data = retriever.invoke(question)

INFO: HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/munna-genai/locations/us-central1/publishers/google/models/text-embedding-005:predict "HTTP/1.1 200 OK"


In [36]:
# lets examine relevant data
print(relevant_data[0].page_content)
print(len(relevant_data))

Acceleration describes how quickly velocity changes with time. An object accelerates when it speeds
4


In [37]:
# lets make our llm a teacher for class 10 student
from langchain_core.prompts import ChatPromptTemplate

teacher_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a very helpful {class} class {subject} teacher. 
            You explain concepts in simple terms.  
            You answer questions based on the context provided. 
            If you don't know the answer, just say 'I don't know'. 
            Don't make up answers. context is {context}""",
        ),

        ("human", "{question}"),
    ]
)

In [38]:
# sample invocation of template
teacher_template.invoke(
    {
        "class": "10",
        "subject": "science",
        "context": relevant_data,
        "question": question,
    }
)

ChatPromptValue(messages=[SystemMessage(content="\n            You are a very helpful 10 class science teacher. \n            You explain concepts in simple terms.  \n            You answer questions based on the context provided. \n            If you don't know the answer, just say 'I don't know'. \n            Don't make up answers. context is [Document(id='e1823750-232e-4de1-bf26-0bfe4cb1e610', metadata={'source': '/Users/munna/VScode/QT-AgenticAI/RAG/hello-langchain/notebooks/concepts/acceleration.txt', 'last_modified': '2026-01-25T16:43:17', 'languages': ['eng'], 'file_directory': '/Users/munna/VScode/QT-AgenticAI/RAG/hello-langchain/notebooks/concepts', 'filename': 'acceleration.txt', 'filetype': 'text/plain', 'parent_id': '4cb0f9bbda343f33247ac3209dae81c8', 'category': 'NarrativeText', 'element_id': '56c2c97180a2609f89953972d9ee47ca'}, page_content='Acceleration describes how quickly velocity changes with time. An object accelerates when it speeds'), Document(id='c70e8bb8-1f91-4

In [39]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(
    model_name="gemini-2.5-flash-lite",
    temperature=0.2,
)

/var/folders/yx/b4_zr4nn6c701qx2kygmf_qc0000gn/T/ipykernel_8116/1510513232.py:3: DeprecationWarning: Use [`ChatGoogleGenerativeAI`][langchain_google_genai.ChatGoogleGenerativeAI] instead.
  llm = ChatVertexAI(
/var/folders/yx/b4_zr4nn6c701qx2kygmf_qc0000gn/T/ipykernel_8116/1510513232.py:3: LangChainDeprecationWarning: The class `ChatVertexAI` was deprecated in LangChain 3.2.0 and will be removed in 4.0.0. An updated version of the class exists in the `langchain-google-genai package and should be used instead. To use it run `pip install -U `langchain-google-genai` and import as `from `langchain_google_genai import ChatGoogleGenerativeAI``.
  llm = ChatVertexAI(
/opt/anaconda3/lib/python3.13/site-packages/google/auth/_default.py:114: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/do

In [40]:
rag_chain = teacher_template | llm

In [41]:
response = rag_chain.invoke(
    {
        "class": "10",
        "subject": "science",
        "context": relevant_data,
        "question": question,
    }
)

/opt/anaconda3/lib/python3.13/site-packages/google/auth/_default.py:114: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [42]:
# rag response
response.pretty_print()

================================== Ai Message ==================================

Acceleration describes how quickly velocity changes with time. An object accelerates when it speeds up, slows down, or changes direction.


In [43]:
# direct llm response
llm.invoke(question).pretty_print()

================================== Ai Message ==================================

Acceleration describes the **rate at which an object's velocity changes over time.**

Here's a breakdown of what that means:

* **Velocity:** Velocity is a measure of both an object's **speed** and its **direction** of motion.
* **Change in Velocity:** This change can happen in a few ways:
    * **Increasing speed:** The object is moving faster.
    * **Decreasing speed:** The object is slowing down (this is often called deceleration, but it's still a form of acceleration).
    * **Changing direction:** Even if the speed remains constant, if the object turns, its velocity changes, and therefore it is accelerating.

**In simpler terms, acceleration tells you how quickly something is speeding up, slowing down, or changing its direction.**

**Key aspects of what acceleration describes:**

* **Magnitude:** The amount of acceleration (how much the velocity is changing per unit of time). This is often measured 